# Paper approach
This notebook implements the method described in the research paper. The approach involves two main stages:

Feature Engineering:

Filtering features:
Selecting the most informative events based on a relevance metric.
Extracting sets of features: Mining recurring sets of events using an estimator of the Longest Common Subsequence (LCSS) algorithm.

Classification:
Building a classifier based on Naive Bayes, (the time-windows and classifiers for each time-window were not implemented since it was advised to put more effort in our own model implementations).

In [2]:
import pandas as pd
import numpy as np
import ast
from collections import Counter
from itertools import combinations
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

Loading data:

In [4]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';')
df.head(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
0,4432881,"[609, 609, 609, 609, 609, 609, 609, 609, 609, ...","[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...",50.876601,4.718143,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",4
1,4432943,"[526, 526, 526, 526, 526, 526, 526, 526, 526, ...","[2744, 4148, 4394, 1566, 1570, 4396, 3634, 412...","[-8573, -8573, -8032, -8032, -8032, -7859, -61...",51.037435,4.431218,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.1,...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",13


Converting `events_sequence` and `vehicles_sequence` from string to list of ints:

In [6]:
df['events_sequence'] = df['events_sequence'].apply(ast.literal_eval)
df['vehicles_sequence'] = df['vehicles_sequence'].apply(ast.literal_eval)

Next calculate total frequency of events in total and frequency of events within each class:

In [8]:
# Total occurrences of each event across all classes
total_event_counts = Counter()
# Occurrences of each event within each class
event_counts_per_class = {}

for incident_type in df['incident_type'].unique():
    event_counts_per_class[incident_type] = Counter()

for idx, row in df.iterrows():
    events = set(row['events_sequence'])  # Use set to avoid duplicate events in the same sequence
    incident_type = row['incident_type']
    total_event_counts.update(events)
    event_counts_per_class[incident_type].update(events)

r is described in the paper as: $$ r = \frac{h_{\text{in class}}}{h_{\text{in all classes}}} $$ 
where ℎ is an event frequency in class or in all classes. This metric is calculated:


In [10]:
relevance_metric = {}

for event in total_event_counts:
    #The maximum h_in class is used
    max_h_in_class = max([event_counts_per_class[cls][event] for cls in event_counts_per_class])
    h_in_all_classes = total_event_counts[event]
    relevance_metric[event] = max_h_in_class / h_in_all_classes

Next we can define a threshold and keep the events equal and over the threshold:

In [12]:
# Set initial threshold
tr = 0.5

selected_events = [event for event, r in relevance_metric.items() if r >= tr]

print(f"Number of events selected with r >= {tr}: {len(selected_events)}")

Number of events selected with r >= 0.5: 635


Next we separate the events in those that were not included

In [14]:
# Events not selected
remaining_events = [event for event in total_event_counts if event not in selected_events]

# Placeholder for events that pass the OaT procedure
additional_events = []

A function is made to calculate the f1 score for a specific subset of the total events

In [41]:
def evaluate_classifier(events_subset):
    # Prepare the data
    df['filtered_events'] = df['events_sequence'].apply(lambda x: [e for e in x if e in events_subset])
    
    # Convert event lists to strings for vectorization
    df['events_str'] = df['filtered_events'].apply(lambda x: ' '.join(map(str, x)))
    
    # Vectorize the events
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['events_str'])
    y = df['incident_type']
    
    # Perform stratified 4-fold cross-validation
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    f1_scores = []
    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        clf = MultinomialNB()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        f1_scores.append(f1)
    
    avg_f1_score = np.mean(f1_scores)
    return avg_f1_score

In [18]:
current_events = selected_events.copy()


Iterating over all events that were filtered out, find the ones that when included , lead to an average f1 score of 0.6 or greater and include them in the `current_events` which are used to make the classification

In [43]:
# Current set of events
current_events = selected_events.copy()

# Evaluate classifier with current events
current_f1_score = evaluate_classifier(current_events)
print(f"Initial F1-score with selected events: {current_f1_score:.4f}")

# Iterate over remaining events
for event in tqdm(remaining_events, desc='OaT Procedure'):
    temp_events = current_events + [event]
    f1 = evaluate_classifier(temp_events)
    if f1 >= 0.6:
        current_events.append(event)
        additional_events.append(event)
        current_f1_score = f1  # Update the current F1-score

print(f"Final F1-score after OaT: {current_f1_score:.4f}")
print(f"Number of additional events added: {len(additional_events)}")

Initial F1-score with selected events: 0.6309


OaT Procedure:  38%|████████████████████████▉                                        | 108/282 [07:44<12:28,  4.30s/it]


KeyboardInterrupt: 

Count total event frequencies and filter out extremely rare events:

In [19]:
# Count event frequencies
event_counts = Counter([event for seq in df['events_sequence'] for event in seq])

# Define a minimum frequency threshold
min_event_frequency = 10  # Adjust based on dataset

# Filter out rare events
frequent_events = set([event for event, count in event_counts.items() if count >= min_event_frequency])

# Update events_sequence by keeping only frequent events
df['filtered_events_sequence'] = df['events_sequence'].apply(lambda seq: [event for event in seq if event in frequent_events])

Prepare the data for frequent itemset mining:

In [21]:
transactions = df['filtered_events_sequence'].tolist()

# Remove empty transactions
transactions = [t for t in transactions if t]

# Use TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_te = pd.DataFrame(te_ary, columns=te.columns_)

Next we calculate the most frequent sequences using fpgrothw algorithm. However, due to huge cost and processing times, `max_length` is set to 2.
Of course, the results will not be optimal, however, it is possible to run the notebook in our laptops

In [31]:
# Apply the FP-Growth algorithm to find frequent itemsets
min_support = 0.05  # Adjust as needed based on dataset size

frequent_itemsets = fpgrowth(df_te, min_support=min_support, use_colnames=True, max_len=2)

print(f"Number of frequent itemsets found: {len(frequent_itemsets)}")
frequent_itemsets.head(5)

Number of frequent itemsets found: 5380


,support,itemsets
0,0.993076,(2708)
1,0.985163,(4026)
2,0.941642,(4066)
3,0.934718,(4068)
4,0.907023,(4148)


Next, we create the columns corresponding to each frequent sequence, checking if in a particular row the frequent sequence appers at least once:

In [33]:
# Convert frequent itemsets into a list of tuples
frequent_sets = [tuple(tup) for tup in frequent_itemsets['itemsets']]

# Function that checks if the exact sequence of events from the tuple is found in the list
def is_tuple_in_list(seq, lst):
    # Convert the tuple to a list to make it easier to check for a subsequence
    seq_list = list(seq)
    # Iterate through the list and check if the subsequence appears
    for i in range(len(lst) - len(seq_list) + 1):
        if lst[i:i+len(seq_list)] == seq_list:
            return 1
    return 0

# Create new features for each frequent itemset in one batch
new_columns = {
    f'itemset_{idx}': df['events_sequence'].apply(lambda x: int(is_tuple_in_list(itemset, x)))
    for idx, itemset in enumerate(frequent_sets)
}

# Add all new features to the DataFrame at once
df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

df.head(5)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type,...,itemset_5370,itemset_5371,itemset_5372,itemset_5373,itemset_5374,itemset_5375,itemset_5376,itemset_5377,itemset_5378,itemset_5379
0,4432881,"[609, 609, 609, 609, 609, 609, 609, 609, 609, ...","[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...",50.876601,4.718143,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",4,...,0,0,0,0,0,0,0,0,0,0
1,4432943,"[526, 526, 526, 526, 526, 526, 526, 526, 526, ...","[2744, 4148, 4394, 1566, 1570, 4396, 3634, 412...","[-8573, -8573, -8032, -8032, -8032, -7859, -61...",51.037435,4.431218,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.1,...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",13,...,0,0,0,0,0,0,0,0,0,0
2,4432955,"[592, 592, 592, 592, 592, 592, 592, 592, 592, ...","[4394, 1566, 1570, 4114, 4168, 4168, 4156, 406...","[-12291, -12291, -12291, -10932, -10932, -1091...",50.864083,4.162115,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, False, True, Tr...",14,...,0,0,0,0,0,0,0,0,0,0
3,4433021,"[576, 576, 576, 576, 576, 576, 576, 576, 576, ...","[4066, 4066, 4066, 4066, 4068, 2742, 4026, 270...","[-14351, -14204, -13890, -13383, -12739, -1243...",51.183220,4.276025,"[0.0, 0.0, 0.0, 0.015625, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",2,...,0,0,0,0,0,0,0,0,0,0
4,4433129,"[634, 634, 634, 634, 634, 634, 634, 634, 634, ...","[4002, 4032, 4028, 2852, 4026, 4110, 2742, 285...","[-224, -224, -223, -222, -222, -222, -220, -22...",50.818727,3.253601,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",14,...,0,0,0,0,0,0,0,0,0,0


# Model evaluation
Now we can evaluate the model on the f1 metric:

In [96]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, LeaveOneOut
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score
from sklearn.dummy import DummyClassifier


In [37]:
# Create a custom scoring function with zero_division parameter
def custom_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted', zero_division=0)

# Wrap the custom scoring function using make_scorer
f1_scorer = make_scorer(custom_f1_score)

In [38]:
target = df.incident_type.copy()
class_counts = target.value_counts()
max_class_count = max(class_counts.values)
sampling_strategy_cross_val = {class_counts.index[i]: int(max_class_count * 0.15) + class_counts.values[i]
                     for i in range(len(class_counts.index)) if class_counts.values[i] < max_class_count}
train_data = df.drop(columns=['incident_id', 'vehicles_sequence', 'events_sequence',
       'seconds_to_incident_sequence', 'approx_lat', 'approx_lon',
       'train_kph_sequence', 'dj_ac_state_sequence', 'dj_dc_state_sequence',
       'incident_type','filtered_events_sequence']).copy()
train_data.head(2)

,itemset_0,itemset_1,itemset_2,itemset_3,itemset_4,itemset_5,itemset_6,itemset_7,itemset_8,itemset_9,...,itemset_5370,itemset_5371,itemset_5372,itemset_5373,itemset_5374,itemset_5375,itemset_5376,itemset_5377,itemset_5378,itemset_5379
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
scores = cross_val_score(MultinomialNB(alpha=1e-1, fit_prior=False),
                         train_data, target,
                        cv=4, scoring=f1_scorer,n_jobs = -1)
print(scores)
print(scores.mean())

C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n


[0.51069358 0.56184167 0.50612084 0.5055799 ]
0.5210589972442037


## Comparison with dummy model
Next we create dummy classifier to see if our model brings some value. With parameter `stratified`, the classifier generates predictions by randomly sampling the classes in a way that respects the class distribution. For example, if the training set contains 70% of samples labeled as `0` and 30% labeled as `1`, the classifier will predict 0 approximately 70% of the time and 1 approximately 30% of the time:

In [118]:
scores = cross_val_score(DummyClassifier(strategy="stratified"),
                         train_data, target,
                        cv=4, scoring=f1_scorer,n_jobs = -1)
print(scores)
print(scores.mean())

C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\stef4\anaconda3\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n


[0.20793466 0.16601656 0.18343041 0.18821455]
0.18639904722815026
